In [1]:
import pickle
from nltk.corpus import stopwords
import re 
import math
import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
ps=PorterStemmer()
import numpy as np

In [2]:
def sbm(query):
    #importing the objects as pickle files
    file1=open('posting_list.pkl',"rb")
    posting_lists=pickle.load(file1)

    file2=open('file_idx.pkl','rb')
    files=pickle.load(file2)

    #set of stopwords
    stpwrds=set(stopwords.words('english'))

    regex = re.compile('[^a-zA-Z\s]')
    words = re.sub(regex,'',query)

    #tokenizing and stemming the query
    tokens = [word.lower() for word in word_tokenize(words)]
    stmd_tkns = [ps.stem(token) for token in tokens if token not in stpwrds]

    #finding which files contain all the query tokens 
    dict_files = {}
    for i in range(len(files)):
        dict_files[i] = 0

    for w in stmd_tkns:
        if w in posting_lists:
            for i in posting_lists[w].keys():
                dict_files[i] += 1

    #Extracting the file names
    results = []
    for i in dict_files.keys():
        if dict_files[i]==len(stmd_tkns):
            results.append(files[i])

    results = sorted(results)

    return results

In [3]:
def tf_idf(query):

    #loading pickle files
    file1 = open('df.pkl','rb')
    df_dict = pickle.load(file1)

    file2 = open('file_idx.pkl','rb')
    file_lst = pickle.load(file2)

    file3 = open('doc_norm.pkl','rb')
    l2_norm_dict = pickle.load(file3)

    file4 = open('doc_words.pkl','rb')
    tot_tkns = pickle.load(file4)

    file5 = open('doc_len.pkl','rb')
    doc_len = pickle.load(file5)

    file6 = open('posting_list.pkl','rb')
    tf_dict = pickle.load(file6)

    #set of stopwords
    stpwrds = set(stopwords.words('english'))

    regex = re.compile('[^a-zA-Z\s]')
    words = re.sub(regex,'',query)

    #tokenizing and stemming the query
    tokens = [word.lower() for word in word_tokenize(words)]
    stmd_tkns = [ps.stem(token) for token in tokens if token not in stpwrds]

    tf_idf_qry = []
    l2_norm = 0

    for i in stmd_tkns:
        tf_idf = 0
        if i in tf_dict:
            tf_idf = stmd_tkns.count(i) * math.log(len(file_lst)/df_dict[i])
        tf_idf_qry.append(tf_idf)
        l2_norm += tf_idf ** 2
    l2_norm = math.sqrt(l2_norm)

    tf_idf_qry = np.array(tf_idf_qry)/l2_norm

    #calculating the score for each document for a given query
    score={}
    for i in range(len(file_lst)):
        tf_idf_doc = []
        for t in stmd_tkns:
            c_tf = 0
            if t in tf_dict and i in tf_dict[t]:
                c_tf = (tf_dict[t][i]) * math.log(len(file_lst)/df_dict[t])
            tf_idf_doc.append(c_tf)
        tf_idf_doc = np.array(tf_idf_doc)/l2_norm_dict[i]
        score[i] = np.dot(tf_idf_qry,tf_idf_doc)

    #sort similarity score in descending order 
    score1=sorted(score.items(),key=lambda x:x[1],reverse=True) 

    result =[]
    #printing the top 15 documents
    count = 15
    for i in score1:
        if count == 0:
            break
        result.append(file_lst[i[0]])
        count-=1

    return result

In [4]:
#loading pickle files
with open('posting_list.pkl','rb') as file1:
    term_f=pickle.load(file1)

with open('df.pkl','rb') as file2:
    idf_dict=pickle.load(file2)

with open('file_idx.pkl','rb') as file3:
    file_lst=pickle.load(file3)

with open('doc_len.pkl','rb') as file4:
    doc_len=pickle.load(file4)

In [5]:
#preprocessing the query
def query_preprocess(query1):
    #set of stopwords
    N=len(file_lst)
    stpwrds = set(stopwords.words('english'))
    tempq = re.compile('[^a-zA-Z\s]')  
    query1 = re.sub(tempq,'',query1)   #except alphabets, remove all other letters
    tokens=nltk.word_tokenize(query1)  #tokenization
    tokens_lwr=[]
    for i in tokens:
        tokens_lwr.append(i.lower())
    tkns=[]
    
    for p in tokens_lwr:               #token stemming
        tkns.append(ps.stem(p))
    tkns1=[]
    
    for p in tkns:                     #remove stopwords
        if p not in stpwrds:
            tkns1.append(p)
    tkns_1=[]
    
    for p in tkns1:                    #remove tokens not present in any of the documents
        if p in idf_dict:
            tkns_1.append(p)
    return tkns_1

In [6]:
def idf(p): #function for finding similarity score
    doc_f=0
    N=len(file_lst)
    if p in idf_dict:
        doc_f1=(len(file_lst)/idf_dict[p])
    idf_val2=math.log((N-doc_f1+0.5)/(doc_f1+0.5))
    return idf_val2

In [7]:
def bm25(query):    
    #set of stopwords
    stpwrds = set(stopwords.words('english'))

    count=0
    N=len(file_lst)
    for i in doc_len:
        count=count+doc_len[i]
    avg_len=count/N
    avg_len

    score={}
    query_pp=query_preprocess(query)
    k=1.2
    b=0.75
    for i in range(len(file_lst)):
            score[i]=0
            for j in query_pp:
                tm_f=0
                if j in term_f:
                    if i in term_f[j]:
                        tm_f=term_f[j][i]
                idf_val=idf(j)
                scr_i=idf_val*(k+1)*tm_f/(tm_f+k*(1-b+b*(doc_len[i]/avg_len)))  #similarity score for document and query
                score[i]+=scr_i

    #sorting similarity score in descending order
    score1=sorted(score.items(),key=lambda x: x[1],reverse=True)

    result = []
    #print the top 15 documents
    count = 15
    for i in score1:
        if count == 0:
            break
        result.append(file_lst[i[0]])
        count-=1
    return result

In [8]:
with open('query.txt','r') as file:
    q_file = file.read()
    file.close()
    
q_dict = {}
for l in q_file.split('\n'):
    if(len(l.split('\t'))==2):
        q_id,query = l.split('\t')
        q_dict[q_id] = query

In [9]:
smb = 'QueryId,Iteration,DocId,Relevance'
tfidf = 'QueryId,Iteration,DocId,Relevance'
bmtf = 'QueryId,Iteration,DocId,Relevance'
for q_id in q_dict:
    qry = q_dict[q_id]
    
    q_list = sbm(qry)
    c = 0
    for i in q_list:
        if c == 15:break
        c += 1
        smb = smb + str('\n'+q_id+','+'1,'+i+','+'1')
    
    k = 0
    while (c<15):
        if file_lst[k] not in q_list:
            smb = smb + str('\n'+q_id+','+'1,'+file_lst[k]+','+'0')
            c += 1
        else:
            k += 1
    
    q_list = tf_idf(qry)
    c = 0
    for i in q_list:
        if c == 15:break
        c += 1
        tfidf = tfidf + str('\n'+q_id+','+'1,'+i+','+'1')
    
    k = 0
    while (c<15):
        if file_lst[k] not in q_list:
            tfidf = tfidf + str('\n'+q_id+','+'1,'+file_lst[k]+','+'0')
            c += 1
        else:
            k += 1 
    
    
    q_list = bm25(qry)
    c = 0
    for i in q_list:
        if c == 15:break
        c += 1
        bmtf = bmtf + str('\n'+q_id+','+'1,'+i+','+'1')
    
    k = 0
    while (c<15):
        if file_lst[k] not in q_list:
            bmtf = bmtf + str('\n'+q_id+','+'1,'+file_lst[k]+','+'0')
            c += 1
        else:
            k += 1

In [10]:
with open('Boolean.csv','w') as file:
    file.write(smb)
    file.close()
with open('Tf-Idf.csv','w') as file:
    file.write(tfidf)
    file.close()
with open('BM25.csv','w') as file:
    file.write(bmtf)
    file.close()